# Stage 3: Hybrid Scoring & Evaluation

이 노트북은 **Stage3 하이브리드 점수 계산 및 평가**를 수행합니다.

### 목적
- **Stage1 (CF-only)**: item2vec 기반 순수 CF 추천
- **Stage1.5 (CF+메타)**: CF + 메타데이터 re-ranking
- **Stage3 (Hybrid)**: CF+메타+오디오 임베딩(0.7/0.3) 결합

위 세 가지 방식의 추천 결과를 동일한 seed 곡에 대해 비교 분석합니다.


In [2]:
# [Cell 1] 공통 import & 설정
# Stage3 Hybrid scoring / Offline eval (Stage1 / Stage1.5 / Stage3-CNN / Stage3-Myna)

import json
import os
import logging
import math
import random
from collections import Counter, defaultdict
from pathlib import Path
from typing import List, Dict, Optional, Union, Tuple

import numpy as np
import pandas as pd
from gensim.models import Word2Vec

# ==========================================
# 공통 상수
# ==========================================
TOPK = 20
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# Pandas 출력 옵션 설정
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 50)

logger.info("✅ 라이브러리 import 완료")
logger.info(f"   - TOPK={TOPK}, RANDOM_SEED={RANDOM_SEED}")


✅ 라이브러리 import 완료
   - TOPK=20, RANDOM_SEED=42


In [3]:
# [Cell 2] 데이터·리소스 로드 (item2vec / song_meta / audio_emb(CNN+Myna) / IN_CATALOG / eval_cases)
# eval_offline_spec(평가기준).md 기준

from tqdm.auto import tqdm

# ==========================================
# 경로 상수 정의 (recommend_model 폴더 기준)
# ==========================================
ROOT_DIR = Path("..")  # recommend_model 폴더 기준 상위 디렉토리
ITEM2VEC_PATH = ROOT_DIR / "models" / "v2_item2vec.model"
SONG_META_PATH = ROOT_DIR / "melon-dataset-excepttar" / "song_meta.json"
TRAIN_JSON_PATH = ROOT_DIR / "melon-dataset-excepttar" / "train.json"

AUDIO_EMB_SOURCES = {
    "cnn": ROOT_DIR / "models" / "audio_embeddings_stage2.npz",
    "myna": ROOT_DIR / "models" / "audio_embeddings_myna_stage2.npz",
}

logger.info("📁 경로 설정:")
logger.info(f"   - ITEM2VEC_PATH: {ITEM2VEC_PATH}")
logger.info(f"   - SONG_META_PATH: {SONG_META_PATH}")
logger.info(f"   - TRAIN_JSON_PATH: {TRAIN_JSON_PATH}")
logger.info(f"   - AUDIO_EMB(CNN): {AUDIO_EMB_SOURCES['cnn']}")
logger.info(f"   - AUDIO_EMB(MYNA): {AUDIO_EMB_SOURCES['myna']}")

# ==========================================
# 1) item2vec 모델 로드
# ==========================================
logger.info("\n🔄 item2vec 모델 로드 중...")
if not ITEM2VEC_PATH.exists():
    raise FileNotFoundError(f"모델 파일이 없습니다: {ITEM2VEC_PATH}")

item2vec_model = Word2Vec.load(str(ITEM2VEC_PATH))
vocab_size = len(item2vec_model.wv)
vector_dim = item2vec_model.wv.vector_size
logger.info(f"✅ item2vec 로드 완료: vocab={vocab_size:,}, dim={vector_dim}")

# In-catalog 곡 집합 (item2vec vocab 기준)
IN_CATALOG_SONGS = {int(k) for k in item2vec_model.wv.key_to_index.keys()}
logger.info(f"📦 IN_CATALOG_SONGS 준비 완료: {len(IN_CATALOG_SONGS):,}곡")

# ==========================================
# 2) song_meta.json 로드 및 lookup 유틸
# ==========================================
logger.info("\n📂 song_meta.json 로드 중...")
if not SONG_META_PATH.exists():
    raise FileNotFoundError(f"메타데이터 파일이 없습니다: {SONG_META_PATH}")

with open(SONG_META_PATH, "r", encoding="utf-8") as f:
    song_meta_list = json.load(f)

song_meta_df = pd.DataFrame(song_meta_list)

if 'id' in song_meta_df.columns:
    song_meta_df.rename(columns={'id': 'song_id'}, inplace=True)

# artist_str 생성
song_meta_df['artist_str'] = song_meta_df['artist_name_basket'].apply(
    lambda x: ", ".join(x) if isinstance(x, list) else str(x)
)

# artist_key 생성 (artist_id_basket 기반)
def get_artist_key(id_basket):
    if isinstance(id_basket, list) and len(id_basket) > 0:
        return str(id_basket[0])
    return "UNKNOWN"

if 'artist_id_basket' in song_meta_df.columns:
    song_meta_df['artist_key'] = song_meta_df['artist_id_basket'].apply(get_artist_key)
else:
    song_meta_df['artist_key'] = "UNKNOWN"

# main_genre 추출
song_meta_df['main_genre'] = song_meta_df['song_gn_gnr_basket'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else "UNK"
)

# issue_year 추출
def parse_year(date_str):
    try:
        return int(str(date_str)[:4])
    except (ValueError, TypeError):
        return np.nan

song_meta_df['issue_year'] = song_meta_df['issue_date'].apply(parse_year)

song_meta_df.set_index('song_id', inplace=True)

cols_to_keep = ['song_name', 'artist_str', 'main_genre', 'issue_year', 'artist_key']
song_meta_df = song_meta_df[cols_to_keep]

logger.info(f"✅ 메타데이터 로드 완료: {len(song_meta_df):,}곡")


def lookup_song_meta(song_id: Union[int, str]) -> Dict:
    """song_id로 메타데이터 조회"""
    try:
        sid = int(song_id)
        if sid in song_meta_df.index:
            return song_meta_df.loc[sid].to_dict()
    except ValueError:
        pass
    return {
        "song_name": None,
        "artist_str": None,
        "main_genre": None,
        "issue_year": None,
        "artist_key": None,
    }

# ==========================================
# 3) 오디오 임베딩 로드 (CNN + Myna)
# ==========================================

def _load_audio_embeddings_npz(path: Path, desc: str) -> Dict[int, np.ndarray]:
    """CNN(구 형식) / Myna(신 형식: song_ids, embeddings) 모두 지원"""
    if not path.exists():
        raise FileNotFoundError(f"오디오 임베딩 파일이 없습니다: {path}")

    data = np.load(path)
    emb_dict: Dict[int, np.ndarray] = {}

    if 'song_ids' in data.files and 'embeddings' in data.files:
        song_ids = data['song_ids']
        embeddings = data['embeddings']
        for i, sid in enumerate(tqdm(song_ids, desc=desc)):
            emb_dict[int(sid)] = embeddings[i]
    else:
        keys = list(data.keys())
        for key in tqdm(keys, desc=desc):
            try:
                sid = int(key)
                emb_dict[sid] = data[key]
            except ValueError:
                continue

    return emb_dict


logger.info("\n🎵 오디오 임베딩 로드 중...")
audio_emb_dict_cnn = _load_audio_embeddings_npz(AUDIO_EMB_SOURCES["cnn"], desc="CNN 임베딩 로드")
audio_emb_dict_myna = _load_audio_embeddings_npz(AUDIO_EMB_SOURCES["myna"], desc="MYNA 임베딩 로드")

cnn_dim = next(iter(audio_emb_dict_cnn.values())).shape[0] if audio_emb_dict_cnn else 0
myna_dim = next(iter(audio_emb_dict_myna.values())).shape[0] if audio_emb_dict_myna else 0

logger.info(f"✅ CNN 임베딩: {len(audio_emb_dict_cnn):,}곡, dim={cnn_dim}")
logger.info(f"✅ MYNA 임베딩: {len(audio_emb_dict_myna):,}곡, dim={myna_dim}")

# (호환용) 기존 코드에서 audio_emb_dict를 참조하는 경우를 위해 기본값 제공
DEFAULT_AUDIO_SOURCE = "cnn"
audio_emb_dict = audio_emb_dict_cnn if DEFAULT_AUDIO_SOURCE == "cnn" else audio_emb_dict_myna

# ==========================================
# 4) eval_cases 생성 (train.json 기반, in-catalog 필터링, 80/10/10 split)
# ==========================================
logger.info("\n🧪 eval_cases 생성 중...")
if not TRAIN_JSON_PATH.exists():
    raise FileNotFoundError(f"train.json 파일이 없습니다: {TRAIN_JSON_PATH}")

with open(TRAIN_JSON_PATH, "r", encoding="utf-8") as f:
    playlists = json.load(f)

logger.info(f"   원본 플레이리스트 개수: {len(playlists):,}")

MIN_LEN = 5

filtered_playlists = []
for idx, pl in enumerate(playlists):
    pid = pl.get("id", idx)
    song_ids = [int(sid) for sid in pl.get("songs", []) if int(sid) in IN_CATALOG_SONGS]
    if len(song_ids) >= MIN_LEN:
        filtered_playlists.append({
            "playlist_id": int(pid),
            "songs": song_ids,
        })

logger.info(f"   in-catalog 필터링 후 플레이리스트 개수: {len(filtered_playlists):,}")

# playlist 단위 split (재현성: RANDOM_SEED)
random.Random(RANDOM_SEED).shuffle(filtered_playlists)

n_total = len(filtered_playlists)
n_train = int(n_total * 0.8)
n_val = int(n_total * 0.1)
train_playlists = filtered_playlists[:n_train]
val_playlists = filtered_playlists[n_train:n_train + n_val]
test_playlists = filtered_playlists[n_train + n_val:]

logger.info(f"   Split 결과 - Train: {len(train_playlists):,}, Val: {len(val_playlists):,}, Test: {len(test_playlists):,}")

# test 플레이리스트로부터 evaluation cases 생성
eval_cases = []
for pl in test_playlists:
    songs = pl["songs"]
    seed = songs[0]
    targets = songs[1:]
    if not targets:
        continue
    eval_cases.append({
        "playlist_id": pl["playlist_id"],
        "seed_song_id": seed,
        "target_song_ids": targets,
    })

logger.info(f"✅ Evaluation cases 생성 완료: {len(eval_cases):,}개")

# 샘플 출력
if eval_cases:
    sample = eval_cases[0]
    seed_meta = lookup_song_meta(sample["seed_song_id"])
    print("\n" + "="*60)
    print("📊 로드 요약")
    print("="*60)
    print(f"  item2vec vocab: {vocab_size:,}곡")
    print(f"  IN_CATALOG_SONGS: {len(IN_CATALOG_SONGS):,}곡")
    print(f"  song_meta: {len(song_meta_df):,}곡")
    print(f"  audio_emb_cnn: {len(audio_emb_dict_cnn):,}곡 (dim={cnn_dim})")
    print(f"  audio_emb_myna: {len(audio_emb_dict_myna):,}곡 (dim={myna_dim})")
    print(f"  eval_cases: {len(eval_cases):,}개")
    print("="*60)
    
    print("\n📋 Sample eval case:")
    print(f"   playlist_id: {sample['playlist_id']}")
    print(f"   Seed: {sample['seed_song_id']} - {seed_meta.get('song_name', 'Unknown')}")
    print(f"   Targets: {len(sample['target_song_ids'])}곡")


📁 경로 설정:
   - ITEM2VEC_PATH: ..\models\v2_item2vec.model
   - SONG_META_PATH: ..\melon-dataset-excepttar\song_meta.json
   - TRAIN_JSON_PATH: ..\melon-dataset-excepttar\train.json
   - AUDIO_EMB(CNN): ..\models\audio_embeddings_stage2.npz
   - AUDIO_EMB(MYNA): ..\models\audio_embeddings_myna_stage2.npz

🔄 item2vec 모델 로드 중...
loading Word2Vec object from ..\models\v2_item2vec.model
loading wv recursively from ..\models\v2_item2vec.model.wv.* with mmap=None
loading vectors from ..\models\v2_item2vec.model.wv.vectors.npy with mmap=None
loading syn1neg from ..\models\v2_item2vec.model.syn1neg.npy with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': '..\\models\\v2_item2vec.model', 'datetime': '2025-12-15T16:29:31.965371', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
✅ item2vec 로드 완료: vocab=316,109, dim=128
📦 

CNN 임베딩 로드:   0%|          | 0/50000 [00:00<?, ?it/s]

MYNA 임베딩 로드:   0%|          | 0/50000 [00:00<?, ?it/s]

✅ CNN 임베딩: 50,000곡, dim=128
✅ MYNA 임베딩: 50,000곡, dim=384

🧪 eval_cases 생성 중...
   원본 플레이리스트 개수: 115,071
   in-catalog 필터링 후 플레이리스트 개수: 113,173
   Split 결과 - Train: 90,538, Val: 11,317, Test: 11,318
✅ Evaluation cases 생성 완료: 11,318개



📊 로드 요약
  item2vec vocab: 316,109곡
  IN_CATALOG_SONGS: 316,109곡
  song_meta: 707,989곡
  audio_emb_cnn: 50,000곡 (dim=128)
  audio_emb_myna: 50,000곡 (dim=384)
  eval_cases: 11,318개

📋 Sample eval case:
   playlist_id: 44898
   Seed: 482358 - Miles Away
   Targets: 35곡


In [4]:
# [Cell 3] Stage1.5 CF+메타 re-ranking 함수 정의
# (기존 Stage1.5 노트북에서 복사한 함수들)

def recommend_cf_raw(model, seed_song_id: Union[int, str], topn: int = 200) -> pd.DataFrame:
    """
    item2vec 모델을 사용하여 유사 곡을 추천하고 메타데이터를 결합합니다.
    (Stage1 순수 CF)
    """
    seed_key = str(seed_song_id)
    
    if seed_key not in model.wv:
        logger.warning(f"⚠️ 곡 ID '{seed_key}'가 모델에 없습니다.")
        return pd.DataFrame()
        
    similar_items = model.wv.most_similar(seed_key, topn=topn+1)
    
    results = []
    for sid_str, score in similar_items:
        if sid_str == seed_key:
            continue
            
        sid = int(sid_str)
        meta = lookup_song_meta(sid)
        
        results.append({
            'song_id': sid,
            'score_cf': float(score),
            'song_name': meta.get('song_name'),
            'artist_str': meta.get('artist_str'),
            'main_genre': meta.get('main_genre'),
            'issue_year': meta.get('issue_year'),
            'artist_key': meta.get('artist_key', 'UNKNOWN')
        })
        
    df = pd.DataFrame(results)
    df = df.head(topn)
    
    return df


def apply_same_artist_penalty(df: pd.DataFrame, 
                            max_per_artist_soft: int = 3, 
                            penalty_per_extra: float = 0.05) -> pd.DataFrame:
    """
    동일 아티스트(artist_key) 곡이 너무 많이 나오면 소프트 페널티를 부여합니다.
    """
    if df.empty:
        return df
        
    df_new = df.copy()
    
    if 'artist_key' not in df_new.columns:
        df_new['artist_key'] = "UNKNOWN"
    
    df_new['artist_key'] = df_new['artist_key'].fillna("UNKNOWN")
    df_new.sort_values('score_cf', ascending=False, inplace=True)
    df_new['artist_order_soft'] = df_new.groupby('artist_key').cumcount()
    
    def calculate_penalty(order):
        if order < max_per_artist_soft:
            return 0.0
        else:
            extra_count = order - max_per_artist_soft + 1
            return extra_count * penalty_per_extra
            
    df_new['artist_penalty_soft'] = df_new['artist_order_soft'].apply(calculate_penalty)
    df_new['score_after_artist'] = df_new['score_cf'] - df_new['artist_penalty_soft']
    
    return df_new


def apply_genre_railguard(df: pd.DataFrame, 
                        seed_main_genre: str, 
                        offrail_penalty_general: float = 0.008,
                        offrail_penalty_special: float = 0.03) -> pd.DataFrame:
    """
    시드 곡과 장르가 다르면 페널티를 부여합니다.
    """
    if df.empty:
        return df
        
    df_new = df.copy()
    
    def get_genre_group(code):
        if not isinstance(code, str) or not code:
            return "UNK"
        if code.startswith("GN07") or code.startswith("GN11"):
            return "TROT"
        if code == "GN1900" or code.startswith("GN19"):
            return "CCM"
        if code == "GN2200" or code.startswith("GN22"):
            return "KIDS"
        if code == "GN2400" or code.startswith("GN24"):
            return "GUGAK"
        if len(code) >= 4:
            return code[:4]
        return "UNK"

    seed_genre_group = get_genre_group(seed_main_genre)
    
    if seed_genre_group == "UNK":
        df_new['genre_penalty'] = 0.0
        df_new['score_after_genre'] = df_new['score_after_artist']
        return df_new

    def calculate_genre_penalty(genre):
        cand_group = get_genre_group(genre)
        if cand_group == seed_genre_group:
            return 0.0
        special_groups = ["TROT", "CCM", "KIDS", "GUGAK"]
        if seed_genre_group in special_groups and cand_group in special_groups:
            return offrail_penalty_special
        if (seed_genre_group in special_groups) != (cand_group in special_groups):
            return offrail_penalty_general * 1.5
        return offrail_penalty_general
        
    df_new['genre_penalty'] = df_new['main_genre'].apply(calculate_genre_penalty)
    df_new['score_after_genre'] = df_new['score_after_artist'] - df_new['genre_penalty']
    
    return df_new


def apply_final_artist_hardcut(df: pd.DataFrame, 
                             topk_final: int = 30, 
                             max_per_artist_final: int = 2) -> pd.DataFrame:
    """
    최종 결과에서 아티스트당 최대 곡 수를 제한합니다 (Hardcut).
    """
    if df.empty:
        return df
    
    if 'artist_key' not in df.columns:
        df = df.copy()
        df['artist_key'] = "UNKNOWN"
        
    if 'score_final' in df.columns:
        df_sorted = df.sort_values('score_final', ascending=False)
    else:
        df_sorted = df
    
    df_sorted = df_sorted.copy()
    df_sorted['artist_key'] = df_sorted['artist_key'].fillna("UNKNOWN")

    selected_rows = []
    artist_counts = {}
    
    for idx, row in df_sorted.iterrows():
        if len(selected_rows) >= topk_final:
            break
        a_key = row.get('artist_key', "UNKNOWN")
        current_count = artist_counts.get(a_key, 0)
        if current_count < max_per_artist_final:
            selected_rows.append(row)
            artist_counts[a_key] = current_count + 1
            
    df_final = pd.DataFrame(selected_rows)
    if not df_final.empty:
        df_final.reset_index(drop=True, inplace=True)
    
    return df_final


def recommend_cf_with_rerank(model, 
                           seed_song_id: Union[int, str], 
                           topn_cf: int = 200, 
                           topk_final: int = 30,
                           max_per_artist_soft: int = 3,
                           max_per_artist_final: int = 2) -> pd.DataFrame:
    """
    Stage1.5: CF 후보 추출 -> 아티스트 페널티(Soft) -> 장르 레일가드 -> 아티스트 하드컷
    
    Returns:
        DataFrame with columns: song_id, song_name, artist_str, main_genre, 
                               issue_year, score_cf, score_final, ...
    """
    # 1. CF 후보 추출
    df = recommend_cf_raw(model, seed_song_id, topn=topn_cf)
    
    if df.empty:
        return df
        
    # 2. Seed 정보 조회
    seed_meta = lookup_song_meta(seed_song_id)
    seed_main_genre = seed_meta.get('main_genre', "UNK")
    
    # 3. Artist Penalty (Soft) 적용
    df = apply_same_artist_penalty(df, 
                                   max_per_artist_soft=max_per_artist_soft,
                                   penalty_per_extra=0.05)
    
    # 4. Genre Railguard 적용
    df = apply_genre_railguard(df, 
                              seed_main_genre=seed_main_genre,
                              offrail_penalty_general=0.008,
                              offrail_penalty_special=0.03)
    
    # 5. 최종 점수 계산 및 정렬
    df['score_final'] = df['score_after_genre']
    df.sort_values(by='score_final', ascending=False, inplace=True)
    
    # 6. Artist Hardcut 적용
    df_final = apply_final_artist_hardcut(df, 
                                          topk_final=topk_final, 
                                          max_per_artist_final=max_per_artist_final)
    
    # 컬럼 정리
    cols = ['song_id', 'song_name', 'artist_str', 'main_genre', 'issue_year', 
            'score_cf', 'score_after_artist', 'score_after_genre', 'score_final']
    cols = [c for c in cols if c in df_final.columns]
    
    return df_final[cols]


logger.info("✅ Stage1.5 re-ranking 함수 정의 완료")


✅ Stage1.5 re-ranking 함수 정의 완료


In [5]:
# [Cell 4] 하이브리드 점수 계산 유틸 함수 정의
# Stage3 Hybrid scoring: CF+메타 점수(0.7) + 오디오 유사도(0.3) 결합

def cosine_sim(a: np.ndarray, b: np.ndarray, eps: float = 1e-8) -> float:
    """
    두 벡터의 코사인 유사도를 계산합니다.
    
    Args:
        a: 첫 번째 벡터
        b: 두 번째 벡터
        eps: 분모 0 방지용 epsilon
        
    Returns:
        코사인 유사도 값 (-1 ~ 1)
    """
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    
    if norm_a < eps or norm_b < eps:
        return 0.0
    
    return float(np.dot(a, b) / (norm_a * norm_b))


def minmax_normalize(arr: np.ndarray) -> np.ndarray:
    """
    1D 배열을 min-max 정규화합니다.
    
    - NaN을 제외한 값들에 대해 0~1 범위로 선형 정규화
    - 모든 값이 같으면(분산 0) → 0.5로 설정
    - NaN 위치는 0.0으로 채움 (오디오 임베딩이 없는 곡은 최소값 취급)
    
    Args:
        arr: 정규화할 1D 배열
        
    Returns:
        정규화된 배열 (0~1 범위)
    """
    arr = np.asarray(arr, dtype=float)
    nan_mask = np.isnan(arr)
    
    # NaN이 아닌 값들만 추출
    valid_vals = arr[~nan_mask]
    
    if len(valid_vals) == 0:
        # 모든 값이 NaN인 경우
        return np.zeros_like(arr)
    
    v_min = valid_vals.min()
    v_max = valid_vals.max()
    
    if v_max - v_min < 1e-8:
        # 모든 valid 값이 동일한 경우 → 0.5로 설정
        result = np.full_like(arr, 0.5)
    else:
        # 정규화
        result = (arr - v_min) / (v_max - v_min)
    
    # NaN 위치는 0.0으로 채움
    result[nan_mask] = 0.0
    
    return result


def rerank_with_audio(df_cfmeta: pd.DataFrame,
                      emb_seed: np.ndarray,
                      audio_emb_dict: Dict[int, np.ndarray],
                      alpha: float = 0.7,
                      beta: float = 0.3) -> pd.DataFrame:
    """
    Stage1.5 결과에 오디오 임베딩 유사도를 결합하여 하이브리드 점수를 계산합니다.
    
    Args:
        df_cfmeta: Stage1.5에서 나온 DataFrame (필수 컬럼: song_id, score_final)
        emb_seed: seed 곡의 오디오 임베딩 벡터
        audio_emb_dict: song_id -> 임베딩 벡터 dict
        alpha: CF+메타 점수 가중치 (기본 0.7)
        beta: 오디오 유사도 가중치 (기본 0.3)
        
    Returns:
        score_hybrid 기준으로 내림차순 정렬된 DataFrame
    """
    if df_cfmeta.empty:
        return df_cfmeta
    
    df = df_cfmeta.copy()
    
    # 1. CF+메타 점수 (score_final을 그대로 사용)
    df['score_cf_raw'] = df['score_final'].copy()
    
    # 2. 각 곡에 대해 오디오 코사인 유사도 계산
    audio_sims = []
    for sid in df['song_id']:
        emb_cand = audio_emb_dict.get(int(sid))
        if emb_cand is not None:
            sim = cosine_sim(emb_seed, emb_cand)
            audio_sims.append(sim)
        else:
            audio_sims.append(np.nan)  # 임베딩이 없으면 NaN
    
    df['score_audio_raw'] = audio_sims
    
    # 3. Min-max 정규화
    df['score_cf_norm'] = minmax_normalize(df['score_cf_raw'].values)
    df['score_audio_norm'] = minmax_normalize(df['score_audio_raw'].values)
    
    # 4. 하이브리드 점수 계산
    df['score_hybrid'] = alpha * df['score_cf_norm'] + beta * df['score_audio_norm']
    
    # 5. score_hybrid 기준 내림차순 정렬
    df.sort_values('score_hybrid', ascending=False, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


logger.info("✅ 하이브리드 점수 계산 함수 정의 완료")
logger.info(f"   - cosine_sim(): 코사인 유사도 계산")
logger.info(f"   - minmax_normalize(): min-max 정규화")
logger.info(f"   - rerank_with_audio(): CF+메타+오디오 하이브리드 재랭킹")


✅ 하이브리드 점수 계산 함수 정의 완료
   - cosine_sim(): 코사인 유사도 계산
   - minmax_normalize(): min-max 정규화
   - rerank_with_audio(): CF+메타+오디오 하이브리드 재랭킹


In [6]:
# [Cell 5] 공통 추천 래퍼 함수 정의 (Stage1 / Stage1.5 / Stage3-CNN / Stage3-Myna)
# eval_offline_spec(평가기준).md 기준 공통 시그니처

TOPN_CF = 200
STAGE3_ALPHA = 0.7
STAGE3_BETA = 0.3
STAGE3_CANDIDATES = 200


def recommend_stage1(seed_song_id: int, topk: int = TOPK) -> List[int]:
    """
    Stage1: item2vec 순수 CF.
    - seed 자기 자신 제외
    - 결과는 IN_CATALOG_SONGS에 포함
    """
    key = str(seed_song_id)
    if key not in item2vec_model.wv:
        return []

    raw = item2vec_model.wv.most_similar(key, topn=max(500, topk + 50))

    rec_ids: List[int] = []
    for sid_str, _ in raw:
        sid = int(sid_str)
        if sid == seed_song_id:
            continue
        if sid not in IN_CATALOG_SONGS:
            continue
        rec_ids.append(sid)
        if len(rec_ids) >= topk:
            break

    return rec_ids


def recommend_stage1_5(seed_song_id: int, topk: int = TOPK) -> List[int]:
    """
    Stage1.5: item2vec + artist/genre re-ranking.
    - 내부적으로 topn_cf 후보를 뽑은 뒤 re-ranking
    """
    df = recommend_cf_with_rerank(
        model=item2vec_model,
        seed_song_id=seed_song_id,
        topn_cf=TOPN_CF,
        topk_final=topk,
    )

    if df.empty:
        return []

    rec_ids = [
        int(sid) for sid in df['song_id'].tolist()
        if int(sid) in IN_CATALOG_SONGS and int(sid) != seed_song_id
    ]
    return rec_ids[:topk]


def _recommend_stage3_with_audio(seed_song_id: int, audio_emb_src: Dict[int, np.ndarray], topk: int = TOPK) -> List[int]:
    """
    Stage3 하이브리드 (CF+메타+오디오) 공통 구현.
    - CF+메타 후보를 넉넉히 뽑은 뒤(STAGE3_CANDIDATES)
    - 오디오 유사도를 결합하여 재랭킹
    """
    df_cfmeta = recommend_cf_with_rerank(
        model=item2vec_model,
        seed_song_id=seed_song_id,
        topn_cf=TOPN_CF,
        topk_final=STAGE3_CANDIDATES,
    )

    if df_cfmeta.empty:
        return []

    emb_seed = audio_emb_src.get(int(seed_song_id))
    if emb_seed is None:
        # seed 임베딩이 없으면 CF+메타 결과로 fallback
        rec_ids = [
            int(sid) for sid in df_cfmeta['song_id'].tolist()
            if int(sid) in IN_CATALOG_SONGS and int(sid) != seed_song_id
        ]
        return rec_ids[:topk]

    df_hybrid = rerank_with_audio(
        df_cfmeta=df_cfmeta,
        emb_seed=emb_seed,
        audio_emb_dict=audio_emb_src,
        alpha=STAGE3_ALPHA,
        beta=STAGE3_BETA,
    )

    rec_ids = [
        int(sid) for sid in df_hybrid['song_id'].tolist()
        if int(sid) in IN_CATALOG_SONGS and int(sid) != seed_song_id
    ]

    return rec_ids[:topk]


def recommend_stage3_cnn(seed_song_id: int, topk: int = TOPK) -> List[int]:
    """Stage3 하이브리드 - CNN 오디오 임베딩 버전"""
    return _recommend_stage3_with_audio(seed_song_id, audio_emb_dict_cnn, topk=topk)


def recommend_stage3_myna(seed_song_id: int, topk: int = TOPK) -> List[int]:
    """Stage3 하이브리드 - Myna 오디오 임베딩 버전"""
    return _recommend_stage3_with_audio(seed_song_id, audio_emb_dict_myna, topk=topk)


# 평가/정성 비교에서 공통으로 사용할 모델 함수 맵
MODEL_FUNCS = {
    "stage1": recommend_stage1,
    "stage1_5": recommend_stage1_5,
    "stage3_cnn": recommend_stage3_cnn,
    "stage3_myna": recommend_stage3_myna,
}

logger.info("✅ 공통 추천 래퍼 준비 완료")
logger.info(f"   - 모델: {list(MODEL_FUNCS.keys())}")
logger.info(f"   - TOPK={TOPK}, TOPN_CF={TOPN_CF}, STAGE3_ALPHA={STAGE3_ALPHA}, STAGE3_BETA={STAGE3_BETA}")


✅ 공통 추천 래퍼 준비 완료
   - 모델: ['stage1', 'stage1_5', 'stage3_cnn', 'stage3_myna']
   - TOPK=20, TOPN_CF=200, STAGE3_ALPHA=0.7, STAGE3_BETA=0.3


In [7]:
# [Cell 6] 평가 지표 함수 정의 (Relevance / Diversity & Coverage / Bias)
# eval_offline_spec(평가기준).md 기준


def recall_at_k(recommended: List[int], target_song_ids: List[int], k: int = TOPK) -> float:
    """
    Recall@K = |Recommended_K ∩ target| / |target|
    (seed별 계산 후 평균)
    """
    targets = set(int(x) for x in target_song_ids)
    if not targets:
        return 0.0

    rec_k = [int(x) for x in recommended[:k]]
    hit = len(set(rec_k) & targets)
    return hit / len(targets)


def ndcg_at_k(recommended: List[int], target_song_ids: List[int], k: int = TOPK) -> float:
    """
    nDCG@K (binary relevance)
    - rel=1 if in target else 0
    """
    targets = set(int(x) for x in target_song_ids)
    if not targets:
        return 0.0

    rec_k = [int(x) for x in recommended[:k]]

    dcg = 0.0
    for i, sid in enumerate(rec_k, start=1):
        rel = 1.0 if sid in targets else 0.0
        if rel > 0:
            dcg += rel / math.log2(i + 1)

    ideal_hits = min(len(targets), k)
    idcg = 0.0
    for i in range(1, ideal_hits + 1):
        idcg += 1.0 / math.log2(i + 1)

    if idcg == 0:
        return 0.0

    return dcg / idcg


def diversity_stats(recommended: List[int], seed_song_id: int, song_meta_df: pd.DataFrame, k: int = TOPK) -> dict:
    """
    TopK 추천 리스트에 대한 다양성/편중 지표.

    Returns:
        {
            "unique_artists": int,
            "unique_genres": int,
            "max_artist_share": float,
            "seed_genre_ratio": float,
        }
    """
    rec_k = [int(x) for x in recommended[:k]]
    if not rec_k:
        return {
            "unique_artists": 0,
            "unique_genres": 0,
            "max_artist_share": 0.0,
            "seed_genre_ratio": 0.0,
        }

    artists = []
    genres = []
    for sid in rec_k:
        if sid in song_meta_df.index:
            row = song_meta_df.loc[sid]
            artists.append(row.get("artist_key", "UNKNOWN"))
            genres.append(row.get("main_genre", "UNK"))

    artist_counts = Counter(artists)
    genre_counts = Counter(genres)

    unique_artists = len(artist_counts)
    unique_genres = len(genre_counts)

    max_artist_share = (max(artist_counts.values()) / k) if artist_counts and k > 0 else 0.0

    # seed 장르 일관성
    seed_main_genre = None
    if int(seed_song_id) in song_meta_df.index:
        seed_main_genre = song_meta_df.loc[int(seed_song_id)]["main_genre"]

    same_genre = 0
    if seed_main_genre is not None:
        for sid in rec_k:
            if sid in song_meta_df.index and song_meta_df.loc[sid]["main_genre"] == seed_main_genre:
                same_genre += 1

    seed_genre_ratio = (same_genre / k) if k > 0 else 0.0

    return {
        "unique_artists": unique_artists,
        "unique_genres": unique_genres,
        "max_artist_share": float(max_artist_share),
        "seed_genre_ratio": float(seed_genre_ratio),
    }


def catalog_coverage(all_recommended: set, in_catalog_songs: set) -> float:
    """coverage@K = (# unique recommended songs) / (# in-catalog songs)"""
    if not in_catalog_songs:
        return 0.0
    return len(all_recommended) / len(in_catalog_songs)


logger.info("✅ 평가 지표 함수 정의 완료")
logger.info("   - recall_at_k(recommended, target_song_ids, k)")
logger.info("   - ndcg_at_k(recommended, target_song_ids, k)")
logger.info("   - diversity_stats(recommended, seed_song_id, song_meta_df, k)")
logger.info("   - catalog_coverage(all_recommended, IN_CATALOG_SONGS)")


✅ 평가 지표 함수 정의 완료
   - recall_at_k(recommended, target_song_ids, k)
   - ndcg_at_k(recommended, target_song_ids, k)
   - diversity_stats(recommended, seed_song_id, song_meta_df, k)
   - catalog_coverage(all_recommended, IN_CATALOG_SONGS)


In [15]:
# [Cell 7] 모델별 오프라인 평가 루프 (Stage1 / Stage1.5 / Stage3-CNN / Stage3-Myna)
# eval_offline_spec(평가기준).md 기준

from tqdm.auto import tqdm


def run_offline_eval(
    eval_cases: List[Dict],
    model_funcs: Dict[str, callable],
    topk: int = TOPK,
    max_cases: Optional[int] = 1000,
) -> pd.DataFrame:
    """
    eval_cases: {"seed_song_id": int, "target_song_ids": list[int], ...} 의 리스트
    model_funcs: {model_name: recommend_fn}

    Returns:
        DataFrame with columns:
        - model
        - recall_at_20
        - ndcg_at_20
        - mean_unique_artists@20
        - mean_unique_genres@20
        - mean_max_artist_share@20
        - mean_seed_genre_ratio@20
        - coverage@20
    """
    if max_cases is not None:
        eval_sample = eval_cases[:max_cases]
    else:
        eval_sample = eval_cases

    rows = []

    for model_name, fn in model_funcs.items():
        recalls = []
        ndcgs = []
        unique_artists_list = []
        unique_genres_list = []
        max_artist_shares = []
        seed_genre_ratios = []

        recommended_union = set()

        for case in tqdm(eval_sample, desc=f"Evaluating {model_name}"):
            seed = int(case["seed_song_id"])
            targets = case.get("target_song_ids", [])

            rec_ids = fn(seed, topk=topk)
            rec_k = rec_ids[:topk]

            # coverage 계산용 union
            recommended_union.update(rec_k)

            # relevance
            recalls.append(recall_at_k(rec_ids, targets, k=topk))
            ndcgs.append(ndcg_at_k(rec_ids, targets, k=topk))

            # diversity/bias
            div = diversity_stats(rec_ids, seed_song_id=seed, song_meta_df=song_meta_df, k=topk)
            unique_artists_list.append(div["unique_artists"])
            unique_genres_list.append(div["unique_genres"])
            max_artist_shares.append(div["max_artist_share"])
            seed_genre_ratios.append(div["seed_genre_ratio"])

        row = {
            "model": model_name,
            "recall_at_20": float(np.mean(recalls)) if recalls else 0.0,
            "ndcg_at_20": float(np.mean(ndcgs)) if ndcgs else 0.0,
            "mean_unique_artists@20": float(np.mean(unique_artists_list)) if unique_artists_list else 0.0,
            "mean_unique_genres@20": float(np.mean(unique_genres_list)) if unique_genres_list else 0.0,
            "mean_max_artist_share@20": float(np.mean(max_artist_shares)) if max_artist_shares else 0.0,
            "mean_seed_genre_ratio@20": float(np.mean(seed_genre_ratios)) if seed_genre_ratios else 0.0,
            "coverage@20": float(catalog_coverage(recommended_union, IN_CATALOG_SONGS)),
        }

        rows.append(row)

    return pd.DataFrame(rows)


# ==========================================
# 평가 실행
# ==========================================
MAX_CASES = 1000  # 속도 테스트용 (필요하면 None 또는 len(eval_cases))
logger.info(f"\n🚀 오프라인 평가 시작: models={list(MODEL_FUNCS.keys())}, TOPK={TOPK}, MAX_CASES={MAX_CASES}")

eval_df = run_offline_eval(eval_cases, MODEL_FUNCS, topk=TOPK, max_cases=MAX_CASES)

print("\n" + "=" * 80)
print("📊 Stage1 / Stage1.5 / Stage3(CNN) / Stage3(Myna) 오프라인 평가 요약")
print("=" * 80)
print(f"   eval_cases used: {min(MAX_CASES, len(eval_cases)):,} / total {len(eval_cases):,}")
print(f"   TOPK = {TOPK}")
print("=" * 80 + "\n")

display(eval_df)



🚀 오프라인 평가 시작: models=['stage1', 'stage1_5', 'stage3_cnn', 'stage3_myna'], TOPK=20, MAX_CASES=1000


Evaluating stage1:   0%|          | 0/1000 [00:00<?, ?it/s]

Evaluating stage1_5:   0%|          | 0/1000 [00:00<?, ?it/s]

Evaluating stage3_cnn:   0%|          | 0/1000 [00:00<?, ?it/s]

Evaluating stage3_myna:   0%|          | 0/1000 [00:00<?, ?it/s]


📊 Stage1 / Stage1.5 / Stage3(CNN) / Stage3(Myna) 오프라인 평가 요약
   eval_cases used: 1,000 / total 11,318
   TOPK = 20



,model,recall_at_20,ndcg_at_20,mean_unique_artists@20,mean_unique_genres@20,mean_max_artist_share@20,mean_seed_genre_ratio@20,coverage@20
0,stage1,0.031292,0.057232,15.011,3.814,0.20180,0.56950,0.054152
1,stage1_5,0.025331,0.050087,16.844,3.280,0.09720,0.67765,0.053874
2,stage3_cnn,0.025150,0.048535,17.020,3.316,0.09675,0.67205,0.053716
3,stage3_myna,0.024958,0.048284,17.133,3.357,0.09635,0.66880,0.053659


In [16]:
# [Cell 7-보조] 결과 해석 가이드 (+ Stage3 CNN vs Myna 비교)

# eval_df는 [Cell 7]에서 생성됨

display(eval_df)

print("\n📋 지표 해석 가이드:")
print("   - recall_at_20: 정답 곡 중 Top20에 포함된 비율 (높을수록 좋음)")
print("   - ndcg_at_20: 정답 곡이 위쪽에 있을수록 높은 점수 (높을수록 좋음)")
print("   - mean_unique_artists@20: Top20 내 평균 아티스트 다양성 (높을수록 다양)")
print("   - mean_unique_genres@20: Top20 내 평균 장르 다양성 (높을수록 다양)")
print("   - mean_max_artist_share@20: 가장 많은 아티스트 점유율 (낮을수록 편중 완화)")
print("   - mean_seed_genre_ratio@20: seed와 같은 장르 비율 (높을수록 일관)")
print("   - coverage@20: in-catalog 곡 중 추천에 한 번이라도 등장한 곡 비율 (높을수록 폭넓게 추천)")

# Stage3 CNN vs Myna 차이 요약
if 'model' in eval_df.columns and set(['stage3_cnn', 'stage3_myna']).issubset(set(eval_df['model'].tolist())):
    cnn_row = eval_df[eval_df['model'] == 'stage3_cnn'].iloc[0]
    myna_row = eval_df[eval_df['model'] == 'stage3_myna'].iloc[0]

    print("\n🆚 Stage3 CNN vs Myna (diff = MYNA - CNN)")
    for col in ['recall_at_20', 'ndcg_at_20', 'mean_unique_artists@20', 'mean_unique_genres@20', 'mean_max_artist_share@20', 'mean_seed_genre_ratio@20', 'coverage@20']:
        if col in eval_df.columns:
            diff = float(myna_row[col]) - float(cnn_row[col])
            print(f"   - {col}: {diff:+.6f}")

# (선택) 결과 파일 저장
# RESULT_PATH = ROOT_DIR / "성능보고서" / "stage간비교" / "v1_eval_results_stage1_stage1_5_stage3_cnn_stage3_myna.csv"
# RESULT_PATH.parent.mkdir(parents=True, exist_ok=True)
# eval_df.to_csv(RESULT_PATH, index=False, encoding="utf-8-sig")
# logger.info(f"📁 결과 저장: {RESULT_PATH}")


,model,recall_at_20,ndcg_at_20,mean_unique_artists@20,mean_unique_genres@20,mean_max_artist_share@20,mean_seed_genre_ratio@20,coverage@20
0,stage1,0.031292,0.057232,15.011,3.814,0.20180,0.56950,0.054152
1,stage1_5,0.025331,0.050087,16.844,3.280,0.09720,0.67765,0.053874
2,stage3_cnn,0.025150,0.048535,17.020,3.316,0.09675,0.67205,0.053716
3,stage3_myna,0.024958,0.048284,17.133,3.357,0.09635,0.66880,0.053659



📋 지표 해석 가이드:
   - recall_at_20: 정답 곡 중 Top20에 포함된 비율 (높을수록 좋음)
   - ndcg_at_20: 정답 곡이 위쪽에 있을수록 높은 점수 (높을수록 좋음)
   - mean_unique_artists@20: Top20 내 평균 아티스트 다양성 (높을수록 다양)
   - mean_unique_genres@20: Top20 내 평균 장르 다양성 (높을수록 다양)
   - mean_max_artist_share@20: 가장 많은 아티스트 점유율 (낮을수록 편중 완화)
   - mean_seed_genre_ratio@20: seed와 같은 장르 비율 (높을수록 일관)
   - coverage@20: in-catalog 곡 중 추천에 한 번이라도 등장한 곡 비율 (높을수록 폭넓게 추천)

🆚 Stage3 CNN vs Myna (diff = MYNA - CNN)
   - recall_at_20: -0.000192
   - ndcg_at_20: -0.000251
   - mean_unique_artists@20: +0.113000
   - mean_unique_genres@20: +0.041000
   - mean_max_artist_share@20: -0.000400
   - mean_seed_genre_ratio@20: -0.003250
   - coverage@20: -0.000057


In [17]:
# [Cell 8] (선택) 통합 평가 결과 CSV 저장

# 결과 파일 경로
RESULT_CSV_PATH = ROOT_DIR / "성능보고서" / "stage간비교" / "v1_eval_results_stage1_stage1_5_stage3_cnn_stage3_myna.csv"

# 폴더 생성
RESULT_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)

# CSV 저장
# - 저장이 필요 없으면 아래 3줄을 주석 처리하세요.
eval_df.to_csv(RESULT_CSV_PATH, index=False, encoding="utf-8-sig")

logger.info(f"\n💾 결과 저장 완료: {RESULT_CSV_PATH}")
print(f"📁 저장 경로: {RESULT_CSV_PATH}")

# 저장된 파일 확인
print("\n📋 저장된 내용 (head):")
print(pd.read_csv(RESULT_CSV_PATH).head(10).to_string(index=False))



💾 결과 저장 완료: C:\Users\ASUS\music_recommend\work\성능보고서\stage간비교\v1_eval_results_stage1_stage1_5_stage3_cnn_stage3_myna.csv


📁 저장 경로: C:\Users\ASUS\music_recommend\work\성능보고서\stage간비교\v1_eval_results_stage1_stage1_5_stage3_cnn_stage3_myna.csv

📋 저장된 내용 (head):
      model  recall_at_20  ndcg_at_20  mean_unique_artists@20  mean_unique_genres@20  mean_max_artist_share@20  mean_seed_genre_ratio@20  coverage@20
     stage1      0.031292    0.057232                  15.011                  3.814                   0.20180                   0.56950     0.054152
   stage1_5      0.025331    0.050087                  16.844                  3.280                   0.09720                   0.67765     0.053874
 stage3_cnn      0.025150    0.048535                  17.020                  3.316                   0.09675                   0.67205     0.053716
stage3_myna      0.024958    0.048284                  17.133                  3.357                   0.09635                   0.66880     0.053659


In [8]:
# [Cell 9] Seed 단일 케이스 정성 비교 (Stage1 / Stage1.5 / Stage3-CNN / Stage3-Myna)

# 🎯 여기에 seed_song_id를 넣고 실행하세요
SEED_SONG_ID = 310545  # 예: 310545 (잔나비 - 로켓트), 원하는 곡으로 변경
TOPK_VIEW = 8


def pretty_print_recommendations_split(seed_song_id: int, topk: int = 10) -> Dict[str, pd.DataFrame]:
    """seed 1개에 대해 모델별 TopK 추천을 4개의 분리된 테이블로 출력"""
    seed_song_id = int(seed_song_id)

    seed_meta = lookup_song_meta(seed_song_id)
    in_catalog = seed_song_id in IN_CATALOG_SONGS
    has_cnn = seed_song_id in audio_emb_dict_cnn
    has_myna = seed_song_id in audio_emb_dict_myna

    print("\n" + "=" * 80)
    print("🎵 Seed 정성 비교")
    print("=" * 80)
    print(f"Seed: [{seed_song_id}] {seed_meta.get('song_name', 'Unknown')} - {seed_meta.get('artist_str', 'Unknown')}")
    print(f"Genre: {seed_meta.get('main_genre', 'UNK')} | In-catalog: {'✅' if in_catalog else '❌'}")
    print(f"Audio Embedding: CNN={'✅' if has_cnn else '❌'} / MYNA={'✅' if has_myna else '❌'}")
    print("=" * 80)

    if not in_catalog:
        print("⚠️ seed_song_id가 IN_CATALOG_SONGS 밖이라 Stage1/1.5/3 평가가 불안정할 수 있습니다.")

    # 모델별 결과를 분리 저장
    model_order = ["stage1", "stage1_5", "stage3_cnn", "stage3_myna"]
    model_labels = {
        "stage1": "📊 Stage1 (CF-only)",
        "stage1_5": "📊 Stage1.5 (CF+Meta)",
        "stage3_cnn": "📊 Stage3-CNN (CF+Meta+Audio)",
        "stage3_myna": "📊 Stage3-Myna (CF+Meta+Audio)",
    }

    result_dfs = {}

    for model_name in model_order:
        fn = MODEL_FUNCS.get(model_name)
        if fn is None:
            continue

        rec_ids = fn(seed_song_id, topk=topk)

        rows = []
        for rank, sid in enumerate(rec_ids[:topk], start=1):
            meta = lookup_song_meta(sid)
            rows.append({
                "rank": rank,
                "song_id": int(sid),
                "song_name": meta.get('song_name', 'Unknown'),
                "artist": meta.get('artist_str', 'Unknown'),
                "genre": meta.get('main_genre', 'UNK'),
            })

        df = pd.DataFrame(rows)
        result_dfs[model_name] = df

        # 각 모델별로 출력
        print(f"\n{model_labels[model_name]}")
        print("-" * 70)
        if df.empty:
            print("  (추천 결과 없음)")
        else:
            display(df)

    return result_dfs


result_tables = pretty_print_recommendations_split(SEED_SONG_ID, topk=TOPK_VIEW)



🎵 Seed 정성 비교
Seed: [310545] 로켓트 - 잔나비
Genre: GN0500 | In-catalog: ✅
Audio Embedding: CNN=✅ / MYNA=✅

📊 Stage1 (CF-only)
----------------------------------------------------------------------


,rank,song_id,song_name,artist,genre
0,1,235997,떠나갈래,블루파프리카,GN0500
1,2,701774,평범한 남자의 유치한 노래 (Feat. 박상욱),김창기,GN0800
2,3,268901,누구를 위하여 종은 울리나,파블로프,GN0500
3,4,53688,햇살 좋은 날,블루파프리카,GN0500
4,5,343891,피난,쏜애플 (THORNAPPLE),GN0500
5,6,84388,Answer,솔루션스 (THE SOLUTIONS),GN0500
6,7,546402,태양,블루파프리카,GN0500
7,8,28032,이사 (Feat. 이장원 Of 페퍼톤스),이선,GN0600



📊 Stage1.5 (CF+Meta)
----------------------------------------------------------------------


,rank,song_id,song_name,artist,genre
0,1,235997,떠나갈래,블루파프리카,GN0500
1,2,268901,누구를 위하여 종은 울리나,파블로프,GN0500
2,3,53688,햇살 좋은 날,블루파프리카,GN0500
3,4,343891,피난,쏜애플 (THORNAPPLE),GN0500
4,5,84388,Answer,솔루션스 (THE SOLUTIONS),GN0500
5,6,701774,평범한 남자의 유치한 노래 (Feat. 박상욱),김창기,GN0800
6,7,143377,백치,쏜애플 (THORNAPPLE),GN0500
7,8,534369,I Don`t Wanna,솔루션스 (THE SOLUTIONS),GN0500



📊 Stage3-CNN (CF+Meta+Audio)
----------------------------------------------------------------------


,rank,song_id,song_name,artist,genre
0,1,343891,피난,쏜애플 (THORNAPPLE),GN0500
1,2,235997,떠나갈래,블루파프리카,GN0500
2,3,53688,햇살 좋은 날,블루파프리카,GN0500
3,4,28032,이사 (Feat. 이장원 Of 페퍼톤스),이선,GN0600
4,5,479667,Motorcycle,우효,GN0500
5,6,268901,누구를 위하여 종은 울리나,파블로프,GN0500
6,7,273724,안드로메다,체리 필터,GN0500
7,8,84388,Answer,솔루션스 (THE SOLUTIONS),GN0500



📊 Stage3-Myna (CF+Meta+Audio)
----------------------------------------------------------------------


,rank,song_id,song_name,artist,genre
0,1,235997,떠나갈래,블루파프리카,GN0500
1,2,53688,햇살 좋은 날,블루파프리카,GN0500
2,3,28032,이사 (Feat. 이장원 Of 페퍼톤스),이선,GN0600
3,4,479667,Motorcycle,우효,GN0500
4,5,343891,피난,쏜애플 (THORNAPPLE),GN0500
5,6,268901,누구를 위하여 종은 울리나,파블로프,GN0500
6,7,273724,안드로메다,체리 필터,GN0500
7,8,84388,Answer,솔루션스 (THE SOLUTIONS),GN0500


In [23]:
# [Cell 10] 플레이리스트 샘플 확인 (train.json + song_meta 조인)

# train.json에서 플레이리스트 5개 샘플 로드
logger.info("\n📋 플레이리스트 샘플 확인 중...")

with open(TRAIN_JSON_PATH, "r", encoding="utf-8") as f:
    all_playlists = json.load(f)

# 플레이리스트 5개 샘플 선택 (길이가 적당한 것들)
sample_playlists = []
for pl in all_playlists:
    songs = pl.get("songs", [])
    if 5 <= len(songs) <= 30:  # 5~30곡 사이의 플레이리스트
        sample_playlists.append(pl)
        if len(sample_playlists) >= 5:
            break

logger.info(f"✅ 샘플 플레이리스트 {len(sample_playlists)}개 선택 완료")

# 각 플레이리스트의 곡들을 song_meta와 조인하여 DataFrame 생성
playlist_dfs = []

for pl in sample_playlists:
    pl_id = pl.get("id", "UNKNOWN")
    pl_title = pl.get("plylst_title", "Unknown")
    pl_tags = ", ".join(pl.get("tags", [])) if pl.get("tags") else "None"
    pl_like_cnt = pl.get("like_cnt", 0)
    songs = pl.get("songs", [])
    
    rows = []
    for rank, song_id in enumerate(songs[:20], start=1):  # 최대 20곡까지만 표시
        song_id_int = int(song_id)
        meta = lookup_song_meta(song_id_int)
        
        rows.append({
            "playlist_id": pl_id,
            "playlist_title": pl_title,
            "tags": pl_tags,
            "like_cnt": pl_like_cnt,
            "rank": rank,
            "song_id": song_id_int,
            "song_name": meta.get("song_name", "Unknown"),
            "artist": meta.get("artist_str", "Unknown"),
            "genre": meta.get("main_genre", "UNK"),
            "issue_year": meta.get("issue_year", None),
        })
    
    if rows:
        df_pl = pd.DataFrame(rows)
        playlist_dfs.append(df_pl)
        
        # 각 플레이리스트별로 출력
        print("\n" + "=" * 80)
        print(f"📋 Playlist ID: {pl_id} | Title: {pl_title}")
        print(f"   Tags: {pl_tags} | Like: {pl_like_cnt} | Total Songs: {len(songs)}곡")
        print("=" * 80)
        display(df_pl)

# 모든 플레이리스트를 하나의 DataFrame으로 합치기 (선택)
if playlist_dfs:
    all_playlists_df = pd.concat(playlist_dfs, ignore_index=True)
    print("\n" + "=" * 80)
    print("📊 전체 플레이리스트 샘플 통합 요약")
    print("=" * 80)
    print(f"   총 플레이리스트 수: {len(playlist_dfs)}개")
    print(f"   총 곡 수: {len(all_playlists_df)}곡")
    print(f"   고유 곡 수: {all_playlists_df['song_id'].nunique()}곡")
    print(f"   고유 아티스트 수: {all_playlists_df['artist'].nunique()}명")
    print(f"   장르 분포:")
    genre_counts = all_playlists_df['genre'].value_counts().head(10)
    for genre, count in genre_counts.items():
        print(f"      - {genre}: {count}곡")
    print("=" * 80)



📋 플레이리스트 샘플 확인 중...
✅ 샘플 플레이리스트 5개 선택 완료



📋 Playlist ID: 61281 | Title: 여행같은 음악
   Tags: 락 | Like: 71 | Total Songs: 19곡


,playlist_id,playlist_title,tags,like_cnt,rank,song_id,song_name,artist,genre,issue_year
0,61281,여행같은 음악,락,71,1,525514,Hey Little Girl,The Sol,GN1400,2013
1,61281,여행같은 음악,락,71,2,129701,Octagon,Royal Bangs,GN0900,2013
2,61281,여행같은 음악,락,71,3,383374,The Road,Honeymoon Suite,GN1000,1991
3,61281,여행같은 음악,락,71,4,562083,Honeymoon,Phoenix,GN0900,2000
4,61281,여행같은 음악,락,71,5,297861,High,James Blunt,GN0900,2005
5,61281,여행같은 음악,락,71,6,139541,"Run, Run, Run",Phoenix,GN0900,2004
6,61281,여행같은 음악,락,71,7,351214,Won`t Stop,OneRepublic,GN1000,2007
7,61281,여행같은 음악,락,71,8,650298,Paris Nights / New York Mornings,Corinne Bailey Rae,GN0900,2010
8,61281,여행같은 음악,락,71,9,531057,Count On Me,Bruno Mars,GN1300,2010
9,61281,여행같은 음악,락,71,10,205238,"Dingue, Dingue, Dingue",Christophe Mae,GN1400,2011



📋 Playlist ID: 76951 | Title: 편하게, 잔잔하게 들을 수 있는 곡.-
   Tags: 까페, 잔잔한 | Like: 17 | Total Songs: 28곡


,playlist_id,playlist_title,tags,like_cnt,rank,song_id,song_name,artist,genre,issue_year
0,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,1,83116,도시의 밤,소울라이츠,GN0400,2008
1,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,2,276692,I`m Alright,소울라이츠,GN0400,2008
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,3,166267,너를 좋아하니까,뷰티핸섬,GN0500,2014
3,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,4,186301,247 (Feat. AMJ),소울사이어티,GN0400,2014
4,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,5,354465,그랬으면 (With 언피),카멜,GN0100,2014
5,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,6,256598,싶어 (Feat. Reddy),오디오라인 (Audioline),GN0400,2015
6,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,7,233195,비숲,"이루펀트, 김박첼라",GN0300,2015
7,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,8,666852,오늘 더 좋아,슈가볼,GN0500,2015
8,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,9,686560,잠든 널 두고 나오며,슈가볼,GN0500,2015
9,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","까페, 잔잔한",17,10,556426,"비행기모드 (Feat. Koonta Of Rude Paper, 오성)",Common Tale,GN0300,2016


...


📋 Playlist ID: 36557 | Title: 멍청이.. 내맘도 몰라.
   Tags: 잔잔한, 추억, 회상 | Like: 5 | Total Songs: 20곡


,playlist_id,playlist_title,tags,like_cnt,rank,song_id,song_name,artist,genre,issue_year
0,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,1,496913,여자는 몰라,FTISLAND (FT아일랜드),GN2500,2008
1,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,2,632529,윤은혜 샐러드송 (샐러드기념일),"윤은혜, 이동건",GN0200,2008
2,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,3,501426,상심증후군 (Feat.Daylight),"노블레스, Daylight",GN0300,2008
3,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,4,515574,You,브라운아이드걸스,GN2500,2008
4,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,5,411161,봄날은 간다,김윤아,GN0600,2001
5,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,6,103411,사랑하기 때문에,왁스,GN0100,2004
6,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,7,178138,너만을 위한 노래,서영은,GN0100,2005
7,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,8,196161,한숨만,J.Lee,GN0400,2005
8,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,9,614003,외쳐본다,이승기,GN0100,2006
9,36557,멍청이.. 내맘도 몰라.,"잔잔한, 추억, 회상",5,10,667106,음악이 싫어졌어 (Feat. 란),노블레스,GN0300,2006



📋 Playlist ID: 10288 | Title: [록/메탈] Written by 이일우
   Tags: 록, Metal, 이일우, M에센셜, 메탈, Rock, 락 | Like: 7 | Total Songs: 15곡


,playlist_id,playlist_title,tags,like_cnt,rank,song_id,song_name,artist,genre,issue_year
0,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,1,613315,Anthem Of Triumph,49 몰핀즈,GN0600,2004
1,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,2,439294,Heart Of Despair,49 몰핀즈,GN0600,2009
2,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,3,230806,Striving Against Fate (Feat. Dokyo13 / 13 Steps),COMBATIVE POST,GN0600,2010
3,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,4,497932,"One Day, One Hope",COMBATIVE POST,GN0600,2010
4,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,5,113479,They Keep Silence (그들은 말이 없다),잠비나이,GN0500,2016
5,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,6,93506,Wake Your Demons,COMBATIVE POST,GN0500,2014
6,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,7,442675,Start A Fire,COMBATIVE POST,GN0500,2014
7,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,8,112831,I`ll Go My Way,COMBATIVE POST,GN0500,2014
8,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,9,146292,소멸의시간 (Time Of Extinction),잠비나이,GN0600,2015
9,10288,[록/메탈] Written by 이일우,"록, Metal, 이일우, M에센셜, 메탈, Rock, 락",7,10,421338,그레이스 켈리 (Grace Kelly),잠비나이,GN0600,2015



📊 전체 플레이리스트 샘플 통합 요약
   총 플레이리스트 수: 5개
   총 곡 수: 94곡
   고유 곡 수: 94곡
   고유 아티스트 수: 74명
   장르 분포:
      - GN0900: 14곡
      - GN0500: 14곡
      - GN0100: 12곡
      - GN0600: 12곡
      - GN1100: 8곡
      - GN0400: 8곡
      - GN2700: 4곡
      - GN0300: 4곡
      - GN1000: 4곡
      - GN1400: 3곡
